In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from brisc.manuscript_analysis import brainrender_3d as br3d
from brisc.manuscript_analysis import overview_image

import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib

matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs

import flexiznam as flz

In [ ]:
# Placeholder
# TODO: setup flexilims project
project = "barcode_diversity_analysis"
flexilims_session = flz.get_flexilims_session(project)
READ_ROOT = (
    flz.get_data_root("processed", flexilims_session=flexilims_session)
    / "rabies_manuscript_figures"
)
SAVE_ROOT = (
    flz.get_data_root("processed", flexilims_session=flexilims_session)
    / "rabies_manuscript_figures"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
reload = False

In [ ]:
# stack_part = overview_image.load_rv_bc_rounds(
#     data_path="becalia_rabies_barseq/BRAC8498.3e/chamber_07"
# )
from iss_preprocess.pipeline.sequencing import basecall_tile

stack, _, _ = basecall_tile(
    "becalia_rabies_barseq/BRAC8498.3e/chamber_07", (5, 2, 7), save_spots=False
)

In [ ]:
stack.shape

In [ ]:
DATA_ROOT = Path("Z:/home/shared/projects")
DATA_BRISC = Path("Y:/")

In [ ]:
# Plot Fig.1
fontsize_dict = {"title": 7, "label": 7, "tick": 5, "legend": 5}
pad_dict = {"label": 1, "tick": 1, "legend": 5}
line_width = 2
line_alpha = 0.9

cm = 1 / 2.54
fig = plt.figure(figsize=(17.4 * cm, 17.4 * cm), dpi=150)
data_path = DATA_ROOT / "becalia_rabies_barseq/BRAC8498.3e/"
save_path = DATA_ROOT / "barcode_diversity_analysis/figures/"

save_fig = False
figname = "fig3"

# 1 In situ seq cartoon
# 2 chambers cartoon
# 3 microscope cartoon
# 4 3D see through brain with rabies imaging site + dorsal flatmap view


ax_3D_brain = fig.add_axes(ax_locations[3])
im = br3d.plot_brainrender_3d(
    ax_3D_brain,
    image_path=data_path / "3D_brain.png",
)

ax_rabies_brain = fig.add_axes(ax_locations[4])
im = overview_image.plot_rabies_raw(
    ax_rabies_brain,
    DATA_BRISC / "processed/becalia_rabies_barseq/BRAC8498.3e/chamber_10/figures/round_overviews/chamber_10_roi_03_barcode_round_1_1_channels_0_1_2_3.png",
    crop_top=0,
    crop_bottom=650,
    crop_left=150,
    crop_right=50,
)

# Plot selected rounds in the new row
axes = [fig.add_axes(loc) for loc in ax_locations[6:]]
overview_image.plot_selected_rounds(
    axes,
    stack,
    window=200,
    selected_rounds=[1, 6, 10],
    fontsize=fontsize_dict["label"],
)


if save_fig:
    pdf_path = f"{save_path}/figures/{figname}.pdf"
    png_path = f"{save_path}/figures/{figname}.png"
    fig.savefig(pdf_path, format="pdf", transparent=True, bbox_inches="tight")
    fig.savefig(png_path, format="png", transparent=True, bbox_inches="tight")
    print(f"Saved PDF figure to {pdf_path}")
    print(f"Saved PNG figure to {png_path}")

In [ ]:
import numpy as np
plt.imshow(np.std(stack, axis=(2,3)))

In [ ]:
np.max(stack[945:1195, 1975:2225, :, :], axis=(0,1,3))

In [ ]:
fig, axes = plt.subplots(1, 4)
overview_image.plot_selected_rounds(
    axes,
    stack[945:1195, 1975:2225, :, :],
    selected_rounds=[1, 2, 9, 10],
    fontsize=fontsize_dict["label"],
    vmin=np.array([0, 0, 0, 0]),
    vmax=np.array([0.5, 0.5, 0.35, 0.35])
)

In [ ]:
fig = plt.figure(figsize=(17.4 * cm, 174 * cm), dpi=150)
import matplotlib.image as mpimg
rabies_img_path = DATA_BRISC / "processed/becalia_rabies_barseq/BRAC8498.3e/chamber_10/figures/round_overviews/chamber_10_roi_03_barcode_round_1_1_channels_0_1_2_3.png"

img = mpimg.imread(rabies_img_path)

ax_rabies_brain = fig.add_axes([0.05, 0.7, 0.35, 0.25])
im = overview_image.plot_rabies_raw(
    ax_rabies_brain,
    img,
    crop_top=0,
    crop_bottom=650,
    crop_left=150,
    crop_right=50,
)

In [ ]:
images_path = DATA_ROOT / "../presentations/becalick_2025"

from iss_preprocess.io import load_stack

rabies_img = load_stack(images_path / "chamber_10_roi_03_barcode_round_1_1_channels_0_1_2_3.ome.tif")
mcherry_img = load_stack(images_path / "chamber_10_roi_03_mCherry_1_channels_0_1_2_3.ome.tif")

In [ ]:
genes_img = load_stack(images_path / "chamber_10_roi_03_genes_round_1_1_channels_0_1_2_3.ome.tif")
hyb_img = load_stack(images_path / "chamber_10_roi_03_hybridisation_round_1_1_channels_0_1_2_3.ome.tif")

In [ ]:
plt.imshow(rabies_img[10000:17000, 11000:17000, 0])

In [ ]:
np.quantile(rabies_img[10000:15000, 11000:15000, :], 0.999, axis=(0,1))

In [ ]:
from iss_preprocess.vis import to_rgb
fig = plt.figure(figsize=(17.4 * cm, 174 * cm), dpi=150)

ax_rabies = fig.add_axes([0.35, 0.07, 0.25, 0.25])
channel_colors = ([1, 0, 0], [0, 1, 0], [1, 0, 1], [0, 1, 1])
rabies_crop = rabies_img[12000:14000, 12500:14000, :]
rgb_stack = to_rgb(
    np.swapaxes(rabies_crop, 0, 1), 
    channel_colors,
    vmin=np.quantile(rabies_crop, 0.9, axis=(0,1)),
    vmax=np.quantile(rabies_crop, 0.999, axis=(0,1)),
)
plt.imshow(rgb_stack)
ax_rabies.invert_yaxis()
plt.axis("off")

mcherry_crop = mcherry_img[12000:14000, 12500:14000, :]
ax_mcherry = fig.add_axes([0.05, 0.07, 0.25, 0.25])

rgb_stack = to_rgb(
    np.swapaxes(mcherry_crop, 0, 1), 
    ([0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 1, 1]),
    vmin=np.quantile(mcherry_crop, 0.8, axis=(0,1)),
    vmax=np.array([1e4, 1e4, 300, 1e4]),
)
plt.imshow(rgb_stack)
ax_mcherry.invert_yaxis()
plt.axis("off")


genes_crop = genes_img[12000:14000, 12500:14000, :]
ax_mcherry = fig.add_axes([0.65, 0.07, 0.25, 0.25])

rgb_stack = to_rgb(
    np.swapaxes(genes_crop, 0, 1), 
    channel_colors,
    vmin=np.quantile(genes_crop, 0.9, axis=(0,1)),
    vmax=np.quantile(genes_crop, 0.999, axis=(0,1)),
)
plt.imshow(rgb_stack)
ax_mcherry.invert_yaxis()
plt.axis("off")

In [ ]:
plt.figure(figsize=(15, 15))
genes_crop = genes_img[8000:8400, 12500:12900, :]
rgb_stack = to_rgb(
    np.swapaxes(genes_crop, 0, 1), 
    channel_colors,
    vmin=np.quantile(genes_crop, 0.9, axis=(0,1)),
    vmax=np.quantile(genes_crop, 0.999, axis=(0,1)),
)
plt.imshow(rgb_stack)


In [ ]:
from iss_preprocess.pipeline.stitch import load_tile_ref_coors
tile_coors = (1, 2, 4)

data_path = "becalia_rabies_barseq/BRAC8498.3e/chamber_08"
stack, _ = load_tile_ref_coors(
    data_path,
    tile_coors,
    "barcode_round",
    filter_r=True,
    projection="max",
    correct_illumination=True,
)

In [ ]:
stack_mcherry, _ = load_tile_ref_coors(
    data_path,
    tile_coors,
    "mCherry_1",
    filter_r=False,
    projection="max",
    correct_illumination=True,
)

In [ ]:
stack_mcherry.shape

In [ ]:
plt.figure(figsize=(15, 15))
plt.subplot(1, 3, 1)
stack_crop = stack[:, :, :, 0]
rgb_stack = to_rgb(
    stack_crop, 
    channel_colors,
    vmin=np.quantile(stack_crop, 0.8, axis=(0,1)),
    vmax=np.quantile(stack_crop, 0.9995, axis=(0,1)),
)
plt.imshow(rgb_stack)
plt.axis("off")

plt.subplot(1, 3, 2)
rgb_stack_mcherry = to_rgb(
    stack_mcherry[:, :, :, 0], 
    ([0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 1, 1]),
    vmin=np.quantile(stack_mcherry[:, :, :, 0], 0.8, axis=(0,1)),
    vmax=np.array([1e4, 1e4, 500, 1e4]),
)
plt.imshow(rgb_stack_mcherry)
ax_mcherry.invert_yaxis()
plt.axis("off")

In [ ]:
plt.figure(figsize=(15, 15))
rgb_stack[:, :, 0] = rgb_stack_mcherry[:, :, 0]
plt.imshow(rgb_stack)

In [ ]:
stack.shape

In [ ]:
stack_std = np.std(stack, axis=(2, 3))
plt.figure(figsize=(15,15))
plt.imshow(stack_std, vmin=np.quantile(stack_std, 0.9), vmax=np.quantile(stack_std, 0.999) )